In [1]:
import numpy as np
import cv2
import json
import pyautogui

# Get the screen width and height
screen_width, screen_height = pyautogui.size()

print(f"Screen Dimensions: {screen_width} x {screen_height}")

# Create a VideoCapture object
cap = cv2.VideoCapture(1)

# Get the screen width and height
screen_width = int(cap.get(3))
screen_height = int(cap.get(4))

# Get the width and height of the camera image
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# JSON file path
json_file_path = "iris_data.json"

# Load existing data from the JSON file
try:
    with open(json_file_path, "r") as json_file:
        json_data = json.load(json_file)
except FileNotFoundError:
    # If the file doesn't exist, initialize an empty dictionary
    json_data = {"circles": [], "left_irises": [], "right_irises": []}


def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Store the position of the clicked circle
        json_data["circles"].append({"x": x, "y": y})

        # Detect face and landmarks (simple face detection without dlib)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        if len(faces) > 0:
            face = faces[0]  # Assuming only one face in the frame

            # Store the positions of left and right irises
            left_eye = (face[0], face[1] + face[3] // 3)
            right_eye = (face[0] + face[2], face[1] + face[3] // 3)

            json_data["left_irises"].append(
                {
                    "x": left_eye[0] / float(width) * screen_width,
                    "y": left_eye[1] / float(height) * screen_height,
                }
            )
            json_data["right_irises"].append(
                {
                    "x": right_eye[0] / float(width) * screen_width,
                    "y": right_eye[1] / float(height) * screen_height,
                }
            )


# Load the pre-trained face cascade classifier from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create a fullscreen window with five circles at corners and center
cv2.namedWindow("Video", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Video", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

cv2.setMouseCallback("Video", mouse_callback)
global rect_color
rect_color = (0, 0, 255)  # Red color

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    circle_radius = 10
    circle_color = (255, 0, 0)  # Blue color

    # Calculate circle positions
    top_left_circle_position = (circle_radius * 2, circle_radius * 2)
    top_center_circle_position = (screen_width // 2, circle_radius * 2)
    top_right_circle_position = (screen_width - circle_radius * 2, circle_radius * 2)
    middle_left_circle_position = (circle_radius * 2, screen_height // 2)
    middle_center_circle_position = (screen_width // 2, screen_height // 2)
    middle_right_circle_position = (
        screen_width - circle_radius * 2,
        screen_height // 2,
    )
    bottom_left_circle_position = (circle_radius * 2, screen_height - circle_radius * 2)
    bottom_center_circle_position = (
        screen_width // 2,
        screen_height - circle_radius * 2,
    )
    bottom_right_circle_position = (
        screen_width - circle_radius * 2,
        screen_height - circle_radius * 2,
    )

    # Draw circles on the frame
    cv2.circle(
        frame, top_center_circle_position, circle_radius, circle_color, -1
    )  # Middle-center
    cv2.circle(
        frame, top_left_circle_position, circle_radius, circle_color, -1
    )  # Top-left
    cv2.circle(
        frame, top_right_circle_position, circle_radius, circle_color, -1
    )  # Top-right
    cv2.circle(
        frame, middle_left_circle_position, circle_radius, circle_color, -1
    )  # Middle-left
    cv2.circle(
        frame, middle_center_circle_position, circle_radius, circle_color, -1
    )  # Middle-center
    cv2.circle(
        frame, middle_right_circle_position, circle_radius, circle_color, -1
    )  # Middle-right
    cv2.circle(
        frame, bottom_left_circle_position, circle_radius, circle_color, -1
    )  # Bottom-left
    cv2.circle(
        frame, bottom_center_circle_position, circle_radius, circle_color, -1
    )  # Bottom-center
    cv2.circle(
        frame, bottom_right_circle_position, circle_radius, circle_color, -1
    )  # Bottom-right

    # Draw a red rectangle at the middle of the frame
    rect_width, rect_height = 180, 280
    rect_start_point = (
        (screen_width - rect_width) // 2,
        (screen_height - rect_height) // 2,
    )
    rect_end_point = (
        (screen_width + rect_width) // 2,
        (screen_height + rect_height) // 2,
    )
    cv2.rectangle(frame, rect_start_point, rect_end_point, rect_color, 2)

    # Detect face and landmarks (simple face detection without dlib)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    if len(faces) > 0:
        landmarks = faces[0]  # Assuming only one face in the frame

        # Check if the face is within the red rectangle
        face_x, face_y, face_w, face_h = landmarks
        if (
            rect_start_point[0] + 10 < face_x < rect_end_point[0] + 10
            and rect_start_point[1] + 10 < face_y < rect_end_point[1] + 10
        ):
            # Face is in the middle, turn the rectangle green
            rect_color = (0, 255, 0)
        else:
            rect_color = (0, 0, 255)

        # Extract eye landmarks
        left_eye = (landmarks[0] + landmarks[2] // 6, landmarks[1] + landmarks[3] // 3)
        right_eye = (landmarks[0] + 5 * landmarks[2] // 6, landmarks[1] + landmarks[3] // 3)

        cv2.circle(frame, left_eye, 2, (255, 255, 255), thickness=1)
        cv2.circle(frame, right_eye, 2, (255, 255, 255), thickness=1)

        left_eye_text = f"Left Eye: ({left_eye[0]}, {left_eye[1]})"
        right_eye_text = f"Right Eye: ({right_eye[0]}, {right_eye[1]})"

        cv2.putText(
            frame,
            left_eye_text,
            (10, 40),
            cv2.FONT_HERSHEY_PLAIN,
            1,
            (255, 255, 255),
            1,
        )
        cv2.putText(
            frame,
            right_eye_text,
            (10, 70),
            cv2.FONT_HERSHEY_PLAIN,
            1,
            (255, 255, 255),
            1,
        )

    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Save the updated data to the JSON file
with open(json_file_path, "w") as json_file:
    json.dump(json_data, json_file, indent=2)

cap.release()
cv2.destroyAllWindows()


Screen Dimensions: 1920 x 1080
